# Новые возможности Python 3.7-3.9

## Python 3.7

### Классы данных (3.7)

https://habr.com/ru/post/415829/

Часто в конструкторе класса можно наблюдать следующую ситуацию:

In [15]:
class SomeThing:
    def __init__(self, value1, value2):
        self.value1 = value1
        self.value2 = value2

В Python 3.7 появились классы данных, в которых не нужно писать все эти однотипные присвоения. Они задаются декоратором `@dataclass`

In [16]:
from dataclasses import dataclass

@dataclass
class SomeThing:
    value1: int
    value2: str
        

s = SomeThing(1, "abc")
s.value1, s.value2

(1, 'abc')

In [17]:
SomeThing.value1

AttributeError: type object 'SomeThing' has no attribute 'value1'

Аннотации типов обязательны, иначе поля игнорируются декоратором

In [19]:
value3 = 5

@dataclass
class SomeThing:
    value1: int
    value2: str
    value3
        

s = SomeThing(1, "abc")
s.value1, s.value2, s.value3

AttributeError: 'SomeThing' object has no attribute 'value3'

### make_dataclass

Библиотека dataclass предоставляет функцию, которая позволяет создавать класс данных следующим образом:

In [20]:
from dataclasses import make_dataclass

SomeThing = make_dataclass("SomeThing", ["value1", "value2"])
s = SomeThing(1, "abc")
s.value1, s.value2

(1, 'abc')

### Значения по умолчанию

В синтаксисе датакласса можно указать значения по умолчанию:

In [21]:
@dataclass
class SomeThing:
    value1: int = 1
    value2: str = "abc"
        
a = SomeThing(45)
b = SomeThing(value2="abcde")

print(a.value1, a.value2)
print(b.value1, b.value2)

45 abc
1 abcde


In [22]:
SomeThing.value1

1

Но надо помнить о том, что аргументы по умолчанию всегда должны следовать после всех позиционных аргументов. Это касается и конструктора, генерируемого декоратором dataclass.

In [23]:
@dataclass
class SomeThing:
    value1: int = 1
    value2: str
        
a = SomeThing(45)
b = SomeThing(value2="abcde")

print(a.value1, a.value2)
print(b.value1, b.value2)

TypeError: non-default argument 'value2' follows default argument

### Frozen Data Class

Отличный способ, чтобы хранить константы: объекты датаклассов, созданных таким образом, неизменяемы.

In [24]:
@dataclass(frozen=True)
class SomeThing:
    value1: int
    value2: str
        
        
s = SomeThing(value1=256, value2="abc")
s.value2 = "abcde"

FrozenInstanceError: cannot assign to field 'value2'

In [25]:
s.value3 = 2

FrozenInstanceError: cannot assign to field 'value3'

### Параметры класса данных

- `init: bool = True` - создать или не создать конструктор
- `repr: bool = True` - создать или не создать `__repr__`
- `eq: bool = True` - создать или не создать метод `__eq__`
- `order: bool = False` - создать или не создать методы сравнения объектов
- `unsafe_hash: bool: False` - создать или не создать метод `__hash__`. Само создание метода зависит также от параметров `eq` и `frozen`
- `frozen: bool = False` - запрет изменения атрибутов класса

Посмотрим, что из себя представляют `__eq__`, `__lt__` и остальные, сгенерированные автоматически

In [26]:
@dataclass(eq=True, order=True, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str

SomeThing(1, 2) == SomeThing(1, 2)

True

In [27]:
SomeThing(1, 2) > SomeThing(2, 1)

False

In [28]:
SomeThing(1, 2) < SomeThing(2, 1)

True

По умолчанию у таких классов сравниваются кортежи значений в том порядке, в котором они были изначально заданы. Посмотреть OrderedDict параметров можно через поле `__dataclass_fields__`

In [29]:
SomeThing(100, "abc").__dataclass_fields__

{'value1': Field(name='value1',type=<class 'int'>,default=<dataclasses._MISSING_TYPE object at 0x0000024BD6617460>,default_factory=<dataclasses._MISSING_TYPE object at 0x0000024BD6617460>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
 'value2': Field(name='value2',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x0000024BD6617460>,default_factory=<dataclasses._MISSING_TYPE object at 0x0000024BD6617460>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD)}

А что с хешами?

In [30]:
@dataclass(eq=True, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
False


In [33]:
@dataclass(eq=False, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
False


In [34]:
@dataclass(frozen=True, eq=True, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
False


In [35]:
@dataclass(frozen=True, eq=False, unsafe_hash=True)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
False


In [40]:
@dataclass(frozen=True, eq=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
True


In [41]:
@dataclass(frozen=False, eq=False, unsafe_hash=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
True


In [42]:
@dataclass(frozen=False, eq=True, unsafe_hash=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

TypeError: unhashable type: 'SomeThing'

In [43]:
@dataclass(frozen=True, eq=True, unsafe_hash=False)
class SomeThing:
    value1: int
    value2: str


print(hash(SomeThing(1, 2)) == hash(SomeThing(1, 2)))
print(hash(SomeThing(1, 2)) == hash(SomeThing(2, 2)))

True
False


### Изменяемые значения по умолчанию

Как мы знаем, использовать изменяемые объекты в качестве значений по умолчанию - плохая идея, поскольку они инстанцируются только один раз при объявлении функции. Dataclass это учитывает:

In [44]:
from typing import List

@dataclass
class SomeThing:
    value: List[int] = []

ValueError: mutable default <class 'list'> for field value is not allowed: use default_factory

Библиотека предлагает использовать default_factory:

In [45]:
from dataclasses import field

@dataclass
class SomeThing:
    value: List[int] = field(default_factory=list)
        
SomeThing().value

[]

#### Параметры field
- `default`: значение по умолчанию. Этот параметр необходим, так как вызов `field` заменяет задание значения поля по умолчанию
- `init`: включает (задан по умолчанию) использование поля в методе `__init__`
- `repr`: включает (задан по умолчанию) использование поля в методе `__repr__`
- `compare`: включает (задан по умолчанию) использование поля в методах сравнения (`__eq__`, `__le__` и других)
- `hash`: может быть булевое значение или None. Если он равен True, поле используется при вычислении хэша. Если указано None (по умолчанию) — используется значение параметра compare.
Одной из причин указать `hash=False` при заданном `compare=True` может быть сложность вычисления хэша поля при том, что оно необходимо для сравнения.
- `metadata`: произвольный словарь или None. Значение оборачивается в MappingProxyType, чтобы оно стало неизменяемым. Этот параметр не используется самими классами данных и предназначен для работы сторонних расширений.


### Обработка после инициализации

В классах данных автоматически создается метод `__init__`, в котором исплоняется код присвоения значений в поля объекта: `self.value = value`. Но что если мы хотим использовать датакласс, но дополнить конструктор какими-то еще действиями? Для этого можем задать метод `__post_init__`

In [46]:
@dataclass
class Book:
    title: str
    author: str
    desc: str = None

    def __post_init__(self):
        self.desc = self.desc or "`%s` by %s" % (self.title, self.author)
        
        
Book("Название", "Автор").desc

'`Название` by Автор'

В этом методе можно использовать дополнительные аргументы конструктора, которые не нужно записывать в self. Для этого предназначен класс `dataclasses.InitVar`:

In [47]:
from dataclasses import InitVar

@dataclass
class Book:
    title: str
    author: str
    gen_desc: InitVar[bool] = True
    desc: str = None

    def __post_init__(self, gen_desc: bool):
        if gen_desc and self.desc is None:
            self.desc = "`%s` by %s" % (self.title, self.author)
            
            
print(Book("Название", "Автор", True).desc)
print(Book("Название", "Автор", False).desc)

`Название` by Автор
None


### Наследование в датаклассах

Если датакласс наследуется от другого датакласса, то он складывает OrderedDict'ы обоих классов и использует полученный общий OrderedDict во всех генерируемых методах.

In [48]:
from typing import Any


@dataclass
class BaseBook:
    title: Any = None
    author: str = None

@dataclass
class Book(BaseBook):
    desc: str = None
    title: str = "Unknown"
        
        
Book()

Book(title='Unknown', author=None, desc=None)

In [49]:
Book().__dataclass_fields__

{'title': Field(name='title',type=<class 'str'>,default='Unknown',default_factory=<dataclasses._MISSING_TYPE object at 0x0000024BD6617460>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
 'author': Field(name='author',type=<class 'str'>,default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x0000024BD6617460>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD),
 'desc': Field(name='desc',type=<class 'str'>,default=None,default_factory=<dataclasses._MISSING_TYPE object at 0x0000024BD6617460>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),_field_type=_FIELD)}

# Новое в Python 3.8

https://habr.com/ru/post/483276/

## Моржовый оператор

Оператор присваивания - как в паскале. Служит для улучшения читаемости кода.

In [50]:
a = 6

# Код ниже присваивает b значение a ** 2
# и проверяет, если b > 0
if (b := a ** 2) > 0: 
    print(f'Квадрат {a} это {b}.') # Квадрат 6 это 36.

Квадрат 6 это 36.


## Только позиционные аргументы

Всё, что записано в сигнатуре функции до знака /, можно передавать только как позиционные аргументы. По имени - нельзя, будет ошибка.

In [51]:
def my_func(a, b, /, c, d, *, e, f):
    return a + b + c + d + e + f
  
my_func(1, 2, 3, 4, 5, 6)         # ошибка: e, f должны быть именованными

TypeError: my_func() takes 4 positional arguments but 6 were given

In [52]:
my_func(a=1, b=2, 3, 4, e=5, f=6) # ошибка: a, b должны быть позиционными

SyntaxError: positional argument follows keyword argument (<ipython-input-52-302744cf128e>, line 1)

In [54]:
my_func(1, 2, 3, d=4, e=5, f=6)   # returns 21

21

In [55]:
my_func(1, 2, c=3, d=4, e=5, f=6)  # returns 21

21

## Улучшенный дебаг print'ом =)

Новый синтаксис f-строк позволяет выводить сразу имя переменной и ее значение:

In [56]:
pi = 3             # В рамках антиковидных ограничений может быть и так

print(f'pi={pi}')  # так мы делали раньше
print(f'{pi=}')    # а так можно делать теперь

pi=3
pi=3


## reversed()

теперь можно вызывать не только над объектами, в которых реализован метод `__reversed__`, но и над теми, в которых описаны `__len__` и `__getitem__`. В числе таких объектов - словари.

## Получение метаданных из других модулей

Новый модуль importlib.metadata позволит получать метаданные (например, версию) из сторонних пакетов.

## Использование continue в блоке finally

In [57]:
for i in range(2):
    try:
        print(i)
    finally:
        print('Тест.')
        continue
        print('Эту строку вы не увидите.')

0
Тест.
1
Тест.


# Новое в Python 3.9
- https://habr.com/ru/post/522170/
- https://habr.com/ru/company/skillfactory/blog/507264/
- https://webdevblog.ru/python-3-9-vse-chto-vam-nuzhno-znat/
- https://proglib.io/p/10-klassnyh-funkciy-python-3-9-2020-11-21

## Объединение словарей
с помощью оператора `|`

In [62]:
a = {'a': 1, 'b': 2}
b = {'c': 3, 'e': 5}
c = a | b
print(c)

{'a': 1, 'b': 3, 'e': 5}


Также можно дозаписывать элементы в существующий словарь:

In [63]:
a = {'a': 1, 'b': 2}
a |= {'c': 3}
print(a)

{'a': 1, 'b': 2, 'c': 3}


У оператора `|=` есть также неожиданное поведение, при котором мы можем справа от него указать итерируемый объект кортежей. Оператор `|` при этом так не может.

In [64]:
a = {'a': 1, 'b': 2}
a |= [('c', 3), ('d', 4)]
print(a)

{'a': 1, 'b': 2, 'c': 3, 'd': 4}


## Generic-подсказки типов в стандартных коллекциях

До Python 3.9 приходилось использовать классы библиотеки typing для того, чтобы описать какие-то сложные типы в аннотациях, например, коллекции. Теперь можно описывать их, используя стандартные коллекции.

In [ ]:
# Python 3.7
from typing import Dict, Tuple, Generator

def func(d: Dict[str, int], t: Tuple[str]) -> Generator:
    return (d[k] for k in t)

In [ ]:
# Python 3.9
from collections.abc import Generator

def func(d: dict[str, int], t: tuple[str]) -> Generator:
    return (d[k] for k in t)

## Расширение возможностей оператора @ в декораторах
Например, теперь можно обращаться к элементам коллекций внутри оператора @

In [60]:
def decorator1(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

def decorator2(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

some = type("cls", (), {})()
some.decorators = [decorator1, decorator2]


@some.decorators[1]
def foo(x, y):
    return x + y

## Строковые методы
Добавилась пара новых методов у строк:

In [65]:
"test_first_try_new".removeprefix("test_")  # удалить подстроку из начала строки

'first_try_new'

In [66]:
"test_first_try_new".removesuffix("_new")   # удалить подстроку из конца строки

'test_first_try'

## Новые возможности асинхронных приложений
При выключении Executor'а раньше футуры, которые еще не начали своё исполнение, всё равно дожидались своей очереди и исполнялись. Теперь их можно отменить с помощью аргумента `cancel_futures` в методе `shutdown` экзекьютора.

In [ ]:
concurrent.futures.Executor.shutdown(cancel_futures=True)
concurrent.futures.ThreadPoolExecutor.shutdown(cancel_futures=True)
concurrent.futures.ProcessPoolExecutor.shutdown(cancel_futures=True)

## Новый парсер
Текущий синтаксический анализатор python реализован на основе грамматики LL (1). Такой парсер считывает документ сверху вниз и слева направо, смотря при этом только на один токен вперед. Это накладывает некоторые ограничения на возможные конструкции языка. Например, такой синтаксис распарсить не получится:

In [ ]:
with (open("file1.txt") as f1, open("file2.txt") as f2):
    pass

Python 3.9 будет использовать синтаксический анализатор на основе грамматики PEG, что позволит формировать более гибкие конструкции в языке. За счет отсутствия необходимости построения промежуточного дерева токенов интерпретация кода может происходить до 10% более эффективно.

## Улучшение производительности
Как и в любой другой версии Python, в версии 3.9 улучшена скорость исполнения кода. На этот раз - за счет применения нового протокола вызова объектов из стандартных коллекций vectorcall.